In [0]:
from pyspark.sql import Row
from pyspark.sql import functions as F

# Unity Catalog target (ueber Widgets konfigurierbar)
dbutils.widgets.text("catalog", spark.sql("SELECT current_catalog()").first()[0])
dbutils.widgets.text("schema", "mini_lakehouse")
CATALOG = dbutils.widgets.get("catalog")
SCHEMA = dbutils.widgets.get("schema")

def tn(name: str) -> str:
    return f"{CATALOG}.{SCHEMA}.{name}"

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")
spark.sql(f"USE SCHEMA {SCHEMA}")

# --- RAW: customers ---
customers_raw = spark.createDataFrame([
    Row(customer_id="C001", country="AT", signup_date="2024-01-15"),
    Row(customer_id="C002", country="DE", signup_date="2024-02-10"),
    Row(customer_id="C003", country="AT", signup_date="2024-03-05"),
    Row(customer_id="C004", country="HU", signup_date="2024-03-20"),
    Row(customer_id="C005", country="DE", signup_date="2024-04-01"),
]).withColumn("signup_date", F.to_date("signup_date"))

# --- RAW: orders ---
orders_raw = spark.createDataFrame([
    Row(order_id="O1001", customer_id="C001", order_ts="2025-01-02 10:10:00", status="paid", amount="120.50"),
    Row(order_id="O1002", customer_id="C001", order_ts="2025-02-05 12:30:00", status="paid", amount="49.99"),
    Row(order_id="O1003", customer_id="C002", order_ts="2025-02-06 09:00:00", status="cancelled", amount="0"),
    Row(order_id="O1004", customer_id="C003", order_ts="2025-02-07 18:20:00", status="paid", amount="15.00"),
    Row(order_id="O1005", customer_id="C004", order_ts="2025-02-07 19:10:00", status="paid", amount="220.00"),
    Row(order_id="O1006", customer_id="C005", order_ts="2025-02-08 08:00:00", status="paid", amount="89.90"),
])

# --- RAW: order_items ---
order_items_raw = spark.createDataFrame([
    Row(order_id="O1001", product_id="P01", qty="1", price="100.50"),
    Row(order_id="O1001", product_id="P02", qty="1", price="20.00"),
    Row(order_id="O1002", product_id="P03", qty="1", price="49.99"),
    Row(order_id="O1004", product_id="P02", qty="1", price="15.00"),
    Row(order_id="O1005", product_id="P01", qty="2", price="110.00"),
    Row(order_id="O1006", product_id="P03", qty="1", price="89.90"),
])

customers_raw.write.mode("overwrite").format("delta").saveAsTable(tn("customers_raw"))
orders_raw.write.mode("overwrite").format("delta").saveAsTable(tn("orders_raw"))
order_items_raw.write.mode("overwrite").format("delta").saveAsTable(tn("order_items_raw"))

print("Created tables:", tn("customers_raw"), tn("orders_raw"), tn("order_items_raw"))
